In [ ]:
import time
import random
import numpy as np

import messages
from simulator import Simulator
from agent import Agent
from unity_env import UnityEnv

In [ ]:
simulator = Simulator(port=5004)
env = UnityEnv(simulator, 17, 2, 1, lambda: np.random.random((2,)))
agent = Agent(env)
#buffer = ReplayBuffer(17, 2, int(1e6))

In [ ]:
simulator.start_server()

In [ ]:
controll_message = messages.ControllMessage(0, 100)
message = messages.RequestMessage("step", controll_message.to_json())

simulator.server.send([message.to_json()])

#response = messages.ResponseMessage.from_json(self.server.recive()[0])
#observation_message = messages.ObservationMessage.from_json(response.value)


In [ ]:
agent.train()

In [ ]:
target_fps = 50
target_frame_duration = 1 / target_fps

max_q_len = 1000
simulator.resume()
time.sleep(0.1)
obs, _, done = simulator.reset()
ep_ret = 0
ep_len = 0

for step in range(1, 1000000+1):
    start_time = time.time()
    if step >= 10000:
        moveInput, turnInput, log_probs = agent.get_action(obs)
    else:
        moveInput, turnInput = random.uniform(-1, 1), random.uniform(-1, 1)

    next_obs, reward, done = simulator.step(moveInput, turnInput)
    ep_ret += reward
    ep_len += 1
    done = False if ep_len==max_q_len else done
    buffer.store(obs, [moveInput, turnInput], reward, next_obs, done)
    obs = next_obs
    
    if done or (ep_len == max_q_len):
        obs, _, done = simulator.reset()
        print(ep_ret)
        ep_ret = 0
        ep_len = 0
        
    #elapsed_time = time.time() - start_time
    #sleep_duration = target_frame_duration - elapsed_time
    #if sleep_duration > 0:
    #    time.sleep(sleep_duration)
        
    if step >= 10000 and step % 50 == 0:
        simulator.pause()
        for _ in range(50):
            batch = buffer.sample_batch()
            agent.update(batch)
            
        simulator.resume()
simulator.pause()

In [ ]:
simulator.shutdown()